In [15]:
import os
import faiss
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Chemins des fichiers
CSV_PATH = r"C:\Users\maram\Desktop\gen\data_text\Animal disease spreadsheet - Sheet1.csv"
PDF_PATH = r"C:\Users\maram\Desktop\gen\data_text\TableauMaladiesChienASSURVETO.pdf"
VECTOR_STORE_PATH = "./vector_store"

# Configuration du modèle
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "llama3"


In [16]:
import pandas as pd
from langchain.schema import Document

def load_and_process_data():
    # Chargement CSV
    data = pd.read_csv(CSV_PATH)
    data.drop(columns=["Treatment", "Advice/ Prevention"], inplace=True)
    
    # Chargement PDF
    loader = PyPDFLoader(PDF_PATH)
    pdf_pages = loader.load()
     # Découpage des textes
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    
    # Combinaison des données
    documents = []
    
    # Ajout des données CSV
    for _, row in data.iterrows():
        content = f"Symptoms: {row['Symptoms']}\nDescription: {row['Description']}"
        documents.append(Document(
            page_content=content,
            metadata={"source": "csv", "disease": row['Recognition']}
        ))
        documents.extend(text_splitter.split_documents(pdf_pages))
    
    return documents
documents = load_and_process_data()

In [17]:
def create_vector_store(docs, force_recreate=False):
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    
    if not force_recreate and os.path.exists(VECTOR_STORE_PATH):
        # Modification ici: ajout du paramètre allow_dangerous_deserialization
        return FAISS.load_local(
            VECTOR_STORE_PATH,
            embeddings,
            allow_dangerous_deserialization=True  # Sécurisé car c'est vous qui avez créé le fichier
        )
    vector_store = FAISS.from_documents(docs, embeddings)
    vector_store.save_local(VECTOR_STORE_PATH)
    return vector_store

vector_store = create_vector_store(documents)
from langchain_community.llms import Ollama

c:\Users\maram\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
PROMPT_TEMPLATE = """[SYSTEM] Vous êtes un assistant vétérinaire expert. Répondez en français.
[CONTEXTE] {context}
[QUESTION] {question}
[INSTRUCTIONS]
1. Analysez les symptômes
2. Proposez un diagnostic probable
3. Indiquez le niveau d'urgence
4. Donnez des conseils de premiers soins"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)
llm = Ollama(model=LLM_MODEL)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
import subprocess

try:
    result = subprocess.run(["ollama", "serve"], check=True, capture_output=True)
    print("Ollama serveur fonctionne correctement")
except subprocess.CalledProcessError as e:
    print(f"Erreur Ollama: {e.stderr.decode()}")


Erreur Ollama: Error: listen tcp 127.0.0.1:11434: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.



C:\Users\maram\AppData\Local\Temp\ipykernel_1628\1116765613.py:14: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=LLM_MODEL)


In [19]:
from langchain_community.llms import Ollama
import torch
def setup_llm():
    try:
        # Essayer d'abord avec CUDA si disponible
        if torch.cuda.is_available():
            llm = Ollama(
                model="llama3",
                temperature=0.7,
                timeout=300
            )
            # Test de connexion
            llm.invoke("Test")
            return llm
    except Exception as e:
        print(f"Erreur avec CUDA: {e}. Basculer sur CPU...")
   
    # Fallback sur CPU
    return Ollama(
        model="llama3",
        temperature=0.7,
        timeout=300
    )
llm = setup_llm()

In [20]:
from langchain_community.llms import Ollama
import os

# Forcer l'utilisation du CPU au niveau système
os.environ["OLLAMA_NO_CUDA"] = "1"

llm = Ollama(
    model="llama3:instruct", 
    temperature=0.7,
    timeout=300
)

In [21]:
def veterinary_chatbot(question):
    try:
        result = qa_chain({"query": question})
        
        print("Réponse:")
        print(result["result"])
        
        print("\nSources utilisées:")
        for doc in result["source_documents"]:
            print(f"\n- Source: {doc.metadata.get('source', 'inconnu')}")
            if 'disease' in doc.metadata:
                print(f"  Maladie: {doc.metadata['disease']}")
            print(f"  Extrait: {doc.page_content[:200]}...")
            
    except Exception as e:
        print(f"Erreur lors de la génération de la réponse: {e}")
        print("Veuillez vérifier que le serveur Ollama est bien lancé et que le modèle est téléchargé.")

# Test avec gestion d'erreur
try:
    veterinary_chatbot("Mon chien vomit et a la diarrhée")
except Exception as e:
    print(f"Erreur critique: {e}")
def veterinary_chatbot(question):
    result = qa_chain({"query": question})
    
    print("Réponse:")
    print(result["result"])
    
    print("\nSources utilisées:")
    for doc in result["source_documents"]:
        print(f"\n- Source: {doc.metadata.get('source', 'inconnu')}")
        if 'disease' in doc.metadata:
            print(f"  Maladie: {doc.metadata['disease']}")
        print(f"  Extrait: {doc.page_content[:200]}...")

# Test
veterinary_chatbot("Mon chien cri")

C:\Users\maram\AppData\Local\Temp\ipykernel_1628\1691426927.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


Réponse:
Bonjour, je suis ravi de vous aider à comprendre la situation de votre chien.

**Analyse des symptômes**
Votre chien vomit et a la diarrhée, ce qui suggère une maladie gastro-entérite. Il est important de noter que les deux formes de cette maladie, gastrique et myocardique, peuvent être graves si elles ne sont pas traitées à temps.

**Diagnostic probable**
La forme gastrique de la maladie peut être suspectée en raison des vomissements et de la diarrhée. Cependant, il est important de consulter un vétérinaire pour obtenir un diagnostic définitif et exclure d'autres causes possibles de ces symptômes.

**Niveau d'urgence**
La situation est considérablement urgente car si votre chien perd trop d'eau et d'électrolytes en raison des diarrhées, il peut entrer en choc circulatoire. Il est donc essentiel de prendre des mesures pour équilibrer les pertes.

**Conseils de premiers soins**
Voici quelques conseils que je vous recommande :

* Faites sortir votre chien de la maison pour évite

In [22]:
import pickle

# Sauvegarde de la QA chain (sans le modèle car il est réinstancié à chaque fois dans l'API)
with open("qa_chain.pkl", "wb") as f:
    pickle.dump(qa_chain, f)

print("✅ Modèle, embeddings, base vectorielle et chaîne QA sauvegardés.")


✅ Modèle, embeddings, base vectorielle et chaîne QA sauvegardés.
